In [66]:
# Cleaning image names
img_dir = '/Volumes/SDD/hipsci_images'

for folder in os.listdir(img_dir):
    
    if '.DS_Store' not in folder: 
        print('Donor: ', folder)
        
        for sub_folder in os.listdir(str(img_dir + '/' + folder)):
            
            if '.DS_Store' not in sub_folder: 
                
                print('Donor', folder, '|| Concentration:', sub_folder)
                
                for image in os.listdir(str(img_dir + '/' + folder + '/' + sub_folder)):
                    
                    imagename = image.split('.')[0]
                    timestamp = imagename.split('y')[1]
                    
                    
                    
                    print image," : ",  timestamp
                    
                    
                    

('Donor: ', 'Bob')
('Donor', 'Bob', '|| Concentration:', 'FN1')
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_19h00m.tif  :  05m20d_19h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_20h00m.tif  :  05m20d_20h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_21h00m.tif  :  05m20d_21h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_22h00m.tif  :  05m20d_22h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_23h00m.tif  :  05m20d_23h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_00h00m.tif  :  05m21d_00h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_01h00m.tif  :  05m21d_01h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_02h00m.tif  :  05m21d_02h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_03h00m.tif  :  05m21d_03h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_04h00m.tif  :  05m21d_04h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_05h00m.tif  :  05m21d_05h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_06h00m.tif  :  05m21d_06h00m
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_07h00m.tif  :  05m21d_07h00m
2014-05-20 HipSci Exp 5_A1

In [40]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
# from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

TRAIN_DIR = '/Volumes/SDD/cells_training_t0t23/cnn_train'
TEST_DIR = '/Volumes/SDD/cells_training_t0t23/cnn_test'
IMG_SIZE = 50 
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

for f in os.listdir(TRAIN_DIR):
    current_file = os.path.basename(f)
    if current_file == '.DS_Store':
        os.remove(f)

OSError: [Errno 2] No such file or directory: '.DS_Store'

In [41]:
import tflearn

In [32]:
def label_img(img):
    label = img.split('.')
    if "L0" in label[0]: 
        return [0,1]
    if "L1" in label[0]:
        return [1, 0]

In [33]:
def create_train_data():
    training_data = []

    for img in os.listdir(TRAIN_DIR):
        
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    
    return training_data



In [34]:
def process_test_data():
    testing_data = []
    
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])

    shuffle(testing_data)
    np.save('test_dat.npy', testing_data)
    return testing_data 

In [35]:
train_data = create_train_data()

In [36]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

# Output = 2, either or normal or abnormal cell
convnet = fully_connected(convnet, 2, activation='softmax')

# set learning rate = LR 
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



In [37]:
#load preloaded model
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded.')

In [39]:
# train data is all data minus last 500
train = train_data[0:500]
# test data is last 500 data
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

ValueError: cannot reshape array of size 500 into shape (50,50,1)

In [24]:
print train[0][0]

[[135 134 134 ..., 126 126 126]
 [134 134 134 ..., 126 126 126]
 [133 134 134 ..., 126 126 126]
 ..., 
 [128 129 129 ..., 128 128 128]
 [129 129 129 ..., 128 128 128]
 [129 129 129 ..., 128 128 128]]
